In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
import scanpy as sc 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import sys
sys.path.append('../../../src')

from spaceoracle.tools.network import RegulatoryFactory
from spaceoracle.gene_factory import GeneFactory

/ihome/djishnu/alw399/.local/lib/python3.9/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound
2025-06-13 13:56:57.711629: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /ihome/crc/install/gcc-12.2.0/postgresql/15.2/lib:/ihome/crc/install/gcc-12.2.0/imagemagick/6.9.12/lib:/ihome/crc/install/gcc-12.2.0/geos/3.9.4/lib:/ihome/crc/install/gcc-12.2.0/zstd/zstd-1.5.5/lib:/ihome/crc/install/gcc-12.2.0/unixodbc/2.3.11/lib:/ihome/crc/install/sodium/1.0.18/lib:/ihome/crc/install/gcc-12.2.0/r/4.4.0/lib64/R/lib:/ihome/crc/install/gcc-12.2

In [3]:
import json 
base_dir = '/ix/djishnu/shared/djishnu_kor11/'

with open(base_dir + 'scGPT_outputs/tonsil_mgs.json', 'r') as f:
    grn = json.load(f)

adata = sc.read_h5ad(base_dir + 'training_data_2025/snrna_human_tonsil.h5ad')
adata

AnnData object with n_obs × n_vars = 5778 × 3333
    obs: 'cell_type', 'author_cell_type', 'cell_type_int', 'banksy_celltypes'
    uns: 'cell_thresholds', 'cell_type_int_colors', 'received_ligands', 'received_ligands_tfl'
    obsm: 'spatial', 'spatial_unscaled'
    layers: 'imputed_count', 'normalized_count'

In [4]:
sp_maps = pd.read_parquet('/ix/djishnu/shared/djishnu_kor11/scGPT_outputs/tonsil_embeddings.parquet')
sp_maps = sp_maps.reindex(adata.obs.index, axis=0).values
sp_maps.shape

(5778, 512)

In [5]:
feature_key = 'scGPT'
adata.obsm['scGPT'] = sp_maps

In [6]:
adata

AnnData object with n_obs × n_vars = 5778 × 3333
    obs: 'cell_type', 'author_cell_type', 'cell_type_int', 'banksy_celltypes'
    uns: 'cell_thresholds', 'cell_type_int_colors', 'received_ligands', 'received_ligands_tfl'
    obsm: 'spatial', 'spatial_unscaled', 'scGPT'
    layers: 'imputed_count', 'normalized_count'

In [7]:
from spaceoracle.astronomer import GeneGeneAstronaut


In [8]:
neil = GeneGeneAstronaut(
    adata=adata,
    annot='cell_type_int', 
    max_epochs=200, 
    learning_rate=5e-3, 
    # spatial_dim=64, # used to create the spatial maps
    batch_size=512,
    grn=grn,
    radius=400,
    contact_distance=50,
    save_dir=base_dir + 'scGPT_runs/tonsil'
)


In [9]:
neil.adata

AnnData object with n_obs × n_vars = 5778 × 3333
    obs: 'cell_type_int'
    obsm: 'spatial', 'spatial_unscaled', 'scGPT'
    layers: 'imputed_count', 'normalized_count'

In [10]:
# %debug 

# We shouldn't use the anchors here, because those are part of our CNN model
neil.run(sp_maps_key='scGPT') 

Using scGPT as spatial maps
Fitting SULT1C2 with 10 modulators
	10 Transcription Factors
	0 Ligand-Receptor Pairs
	0 TranscriptionFactor-Ligand Pairs
Early stopping triggered at epoch 86
0.9333
Using scGPT as spatial maps
Fitting ANO4 with 6 modulators
	6 Transcription Factors
	0 Ligand-Receptor Pairs
	0 TranscriptionFactor-Ligand Pairs
Early stopping triggered at epoch 52
-0.0030
Using scGPT as spatial maps
Fitting TCF7 with 16 modulators
	16 Transcription Factors
	0 Ligand-Receptor Pairs
	0 TranscriptionFactor-Ligand Pairs
Early stopping triggered at epoch 188
0.9836
Using scGPT as spatial maps
Fitting INSR with 10 modulators
	10 Transcription Factors
	0 Ligand-Receptor Pairs
	0 TranscriptionFactor-Ligand Pairs
0.9172
Using scGPT as spatial maps
Fitting ADGRE5 with 9 modulators
	9 Transcription Factors
	0 Ligand-Receptor Pairs
	0 TranscriptionFactor-Ligand Pairs
0.9215
Using scGPT as spatial maps
Fitting PRODH2 with 10 modulators
	10 Transcription Factors
	0 Ligand-Receptor Pairs
	0 

KeyboardInterrupt: 